### n개의 시트 통합하여 집계, 차트 생성  
1.numpy,list활용 방법  
    - 데이터통합 로직 복잡  
2.df활용방법  
 

In [ ]:
from openpyxl import *
from openpyxl.chart import *
import numpy as np
import datetime

data_buffer1=[]
data_buffer2=[[]for i in range(2)]  #중첩리스트

for i in range(2):
    file_name='../../testData/rpatest'+str(i+1)+'.xlsx'
    book=load_workbook(file_name,data_only=True)  #데이터값으로 읽기
    sheet=book.active
    cells=sheet['a4:b13']
    for c1,c2 in cells:
        if i==0:
            data_buffer1.append(c1.value)  #'상품명'
        data_buffer2[i].append(c2.value)   #'매출'
del book
del sheet

book=Workbook()
sheet=book.active
data_buffer1=np.array(data_buffer1[:])
data_buffer2=np.array(data_buffer2[:])
rows=[]
#부서별로 된 파일을 하나의 시트로 합침..상품수=row, 부서별실적=column
for i in range(len(data_buffer1)): #상품수 loop
    dummy=[]
    dummy.append(data_buffer1[i].tolist())
    dummy.extend([data_buffer2[0][i].tolist(),data_buffer2[1][i].tolist()]) #리스트확장
    max_val=np.max([data_buffer2[0][i],data_buffer2[1][i]]).tolist()
    dummy.append(max_val)
    mean_val=np.mean([data_buffer2[0][i],data_buffer2[1][i]]).tolist()
    dummy.append(mean_val)
    rows.append(dummy)  #로우 리스트에 추가
    
sheet.append(['성품명','1팀','2팀','최대','평균'])

for row in rows:
    sheet.append(row)

    
data=Reference(sheet,min_col=2,max_col=3,min_row=1,max_row=11) #차트데이터
categs=Reference(sheet,min_col=1,min_row=2,max_row=11) #범주

chart=BarChart()
chart.add_data(data,titles_from_data=True)
chart.set_categories(categs)

#chart.legend=None
chart.y_axis.majorGridLines=None
chart.varyColors=True
chart.title='타이틀'
sheet.add_chart(chart,'a12')

now= datetime.datetime.now()
nowDate= now.strftime('%Y-%m-%d')
file_name= str(nowDate)+'sum.xlsx'
print(file_name)
book.save(file_name)

### df활용

In [ ]:
from openpyxl import *
from openpyxl.chart import *
import numpy as np
import datetime
import pandas as pd

data_buffer1=[]
data_buffer2=[[]for i in range(2)]  #중첩리스트
dfs=pd.DataFrame()

for i in range(2):
    file_name='../../testData/rpatest'+str(i+1)+'.xlsx'
    df = pd.read_excel(file_name)
    col_nm=str(i+1)+'팀'
    if i==0:
        dfs=df
    else:
        dfs=pd.merge(dfs,df,how='outer',on='상품명')
        
dfs['ma']=dfs.iloc[:,1:3].apply(np.max,axis=1)
dfs['me']=dfs.iloc[:,1:3].apply(np.mean,axis=1)

book=Workbook()
sheet=book.active

#시트에 복사
sheet.append(['성품명','1팀','2팀','최대','평균'])
for seq,row in dfs.iterrows():
    sheet.append(list(row))

#차트
data=Reference(sheet,min_col=2,max_col=3,min_row=1,max_row=11) #차트데이터
categs=Reference(sheet,min_col=1,min_row=2,max_row=11) #범주

chart=BarChart()
chart.add_data(data,titles_from_data=True)
chart.set_categories(categs)

#chart.legend=None
chart.y_axis.majorGridLines=None
chart.varyColors=True
chart.title='타이틀'
sheet.add_chart(chart,'a12')

now= datetime.datetime.now()
nowDate= now.strftime('%Y-%m-%d')
file_name= '../../testData/'+str(nowDate)+'sum.xlsx'
print(file_name)
book.save(file_name)

In [ ]:
df1.상품명=df1.상품명.replace('상품1','2')

In [5]:
import pandas as pd
i=0
file_name='../testData/rpatest'+str(i+1)+'.xlsx'
df2 = pd.read_excel(file_name)

In [ ]:
pd.merge(df1[['상품명','판매량']],df2[['상품명','판매량']],how='outer',on='상품명')

In [8]:
df2

,상품명,판매량
0,상품1,100
1,상품2,120
2,상품3,140
3,상품4,160
4,상품5,180
5,상품6,200
6,상품7,220
7,상품8,240
8,상품9,260
9,상품10,280


In [14]:
df2.rolling(5).mean()

,판매량
0,NaN
1,NaN
2,NaN
3,NaN
4,140.0
5,160.0
6,180.0
7,200.0
8,220.0
9,240.0
